# **Web Scraping | SQL**

### **SET-UP**

In [1]:
# import the libraries
import pandas as pd
import json
import datetime
import requests
import sqlite3
from bs4 import BeautifulSoup

**Web Scraping:** Job offers' Website (Luxembourgish)
https://en.jobs.lu/ 

**The objective:** The goal is to parse the HTLM of the site in order to identify and separate the most relevant information. This output will be transformed into a JSON and all data will be stored in BD SQL.

**Filter:** IT / Programming

In [2]:
# Setting-up the environment
def get_data(url, parse):
    raw_html = requests.get(url, parse).content
    return BeautifulSoup(raw_html)

# Identifying the path
url = 'https://en.jobs.lu/'
raw_html = requests.get(url).content
html = BeautifulSoup(raw_html) # Read HTLM and other formats as well as generate data tree

# Extraction of the information
html = get_data('https://en.jobs.lu/it_programming_jobs.aspx', 'html.parse')
job_offers = html.find_all("article", class_="job-list-item")

# Creating dictionary
my_jobs = {}
for index in range(len(job_offers)):
    job_n = "job_{}".format(index + 1) 
    
    # Pré-process and data cleaning
    today1 = datetime.datetime.today().strftime('%d/%m/%Y') # Conversion to display the day of each publication in a date format (publication day instead of "Today")
    yesterday1 = datetime.datetime.today() - datetime.timedelta(days=1) # Conversion to datetime in order to present the day of each publication (cont.)
    yesterday1 = yesterday1.strftime('%d/%m/%Y') # (cont.) in date format (previous day instead of "yesterday")
    my_jobs[job_n] = {
    "Date" : job_offers[index].find_all("span", class_ = "date")[0].get_text().strip().replace("Today", today1).replace("1 Day Ago", yesterday1),
    "Job" : job_offers[index].find_all("span", class_ = "job")[0].get_text().strip('\n'),
    "Company" : job_offers[index].find_all("span", class_ = "company")[0].get_text().strip('\n'),
    "Location" : str(job_offers[index].find_all("span", class_ = "location")[0].get_text().replace(","," ").rstrip("LU").lstrip('1234567890_F-').strip().capitalize())
    }

In [3]:
#my_jobs
df_jobs = pd.DataFrame(my_jobs).T # Displaying results in a dataframe
df_jobs # Calling dataframe

,Date,Job,Company,Location
job_1,25/08/2020,Application Operational Support Analyst for di...,Deloitte General Services,Luxembourg
job_2,25/08/2020,Senior JAVA Engineer,Finologee S.A.,Leudelange
job_3,25/08/2020,ISO Coordinator and Implementor,Groupe Wagner Luxembourg,Cloche d'or luxembourg
job_4,25/08/2020,Project Officer,Université de Luxembourg,Belval
job_5,25/08/2020,IT Security Specialist,Groupe Wagner Luxembourg,Cloche d'or luxembourg
job_6,25/08/2020,Life Cycle Assessment Analyst with proven Prog...,Luxembourg Institute of Science and Technology...,Belval
job_7,25/08/2020,Ingénieur DevOps (m/f),Bâloise Assurance Luxembourg,Bertrange luxembourg
job_8,25/08/2020,Junior Systems and Storage Technician (m/f),BCE Broadcasting Center Europe S.A.,L1543 luxembourg
job_9,25/08/2020,IT Admin System Manager,Link Point SA,Luxembourg
job_10,25/08/2020,Microsoft SQL Server Administrator (f/m/d),Dennemeyer,Howald


# **SQL**
#### Storage

In [4]:
# Creation of the BD file, creation of the table, configuration and categorization of attributes
db_file = "it_jobs.sqlite.db" 
conn = sqlite3.connect(db_file) # Starts connection
cur = conn.cursor() # Cursor definition that allows manipulating and navigating with the DB
# Creating table
query = """ CREATE TABLE IF NOT EXISTS jobsit ( 
                                    date datetime,
                                    job text,
                                    company text,
                                    location text
                                ); """
cur.execute(query) # Creating table, runs the query

In [5]:
conn.commit() # Recording the table in the database
df_jobs = pd.DataFrame(my_jobs).T # Creating dataframe
df_jobs.to_sql("jobsit", con=conn, if_exists="append", index=False) # Setting parameters and filling the table with data
df_jobs

,Date,Job,Company,Location
job_1,25/08/2020,Application Operational Support Analyst for di...,Deloitte General Services,Luxembourg
job_2,25/08/2020,Senior JAVA Engineer,Finologee S.A.,Leudelange
job_3,25/08/2020,ISO Coordinator and Implementor,Groupe Wagner Luxembourg,Cloche d'or luxembourg
job_4,25/08/2020,Project Officer,Université de Luxembourg,Belval
job_5,25/08/2020,IT Security Specialist,Groupe Wagner Luxembourg,Cloche d'or luxembourg
job_6,25/08/2020,Life Cycle Assessment Analyst with proven Prog...,Luxembourg Institute of Science and Technology...,Belval
job_7,25/08/2020,Ingénieur DevOps (m/f),Bâloise Assurance Luxembourg,Bertrange luxembourg
job_8,25/08/2020,Junior Systems and Storage Technician (m/f),BCE Broadcasting Center Europe S.A.,L1543 luxembourg
job_9,25/08/2020,IT Admin System Manager,Link Point SA,Luxembourg
job_10,25/08/2020,Microsoft SQL Server Administrator (f/m/d),Dennemeyer,Howald


In [6]:
conn.close() # Disconnect db connection

In [7]:
conn = sqlite3.connect(db_file) # Starts connection
cur = conn.cursor() # Cursor definition that allows manipulating and navigating with the DB
query1 = """
    SELECT * 
    FROM jobsit
    WHERE location="Luxembourg"
    OR company="FYTE"
"""
cur.execute(query1) # Runs the query
call = cur.fetchall() # Getting results
show = pd.DataFrame(call) # Creates DataFrame
show # Shows DataFrame

,0,1,2,3
0,30/03/2020,Information Security Officer (H/F) en CDI,AXA Assurances Luxembourg,Luxembourg
1,30/03/2020,Senior Cloud Architect (Azure),The Recruiter Sàrl,Luxembourg
2,30/03/2020,urgent ! .Net developpeur WPF,Fast Recruitment,Luxembourg
3,30/03/2020,IT Junior Trainer and Support Analyst (m/f),Clifford Chance,Luxembourg
4,30/03/2020,System Engineer - VMware,The Recruiter Sàrl,Luxembourg
...,...,...,...,...
78,24/08/2020,Security and Licencing Analyst (m/f),BCE Broadcasting Center Europe S.A.,Luxembourg
79,24/08/2020,Scrum Master (H/F) - CDD,Sogelife S.A.,Luxembourg
80,24/08/2020,Développeur confirmé PHP/symfony fullstack (H/F),CockpitLAB,Luxembourg
81,24/08/2020,Data Base Administrator (m/f/d),European Depositary Bank SA,Luxembourg


In [8]:
conn.close() # Disconnect db connection

In [9]:
show

,0,1,2,3
0,30/03/2020,Information Security Officer (H/F) en CDI,AXA Assurances Luxembourg,Luxembourg
1,30/03/2020,Senior Cloud Architect (Azure),The Recruiter Sàrl,Luxembourg
2,30/03/2020,urgent ! .Net developpeur WPF,Fast Recruitment,Luxembourg
3,30/03/2020,IT Junior Trainer and Support Analyst (m/f),Clifford Chance,Luxembourg
4,30/03/2020,System Engineer - VMware,The Recruiter Sàrl,Luxembourg
...,...,...,...,...
78,24/08/2020,Security and Licencing Analyst (m/f),BCE Broadcasting Center Europe S.A.,Luxembourg
79,24/08/2020,Scrum Master (H/F) - CDD,Sogelife S.A.,Luxembourg
80,24/08/2020,Développeur confirmé PHP/symfony fullstack (H/F),CockpitLAB,Luxembourg
81,24/08/2020,Data Base Administrator (m/f/d),European Depositary Bank SA,Luxembourg


---

**Author**: Gonçalo Guimarães Gomes. Portuguese. Data Analyst and Digital Brand Marketer. Postgraduate in Data Science and in Digital Marketing. Degree and Executive Master in Marketing Management.

### **Contacts**

- [Linkedin](https://www.linkedin.com/in/goncaloggomes/)
- [Twitter](https://twitter.com/goncaloggomes)
- [Medium Profile](https://medium.com/@goncaloggomes)
- [GitHub](https://github.com/goncaloggomes)
- [Email](mailto:goncaloggomes@gmail.com)